In [248]:
import pandas as pd

df =pd.read_csv('dataset/synthetic_logs_new.csv')
df

,timestamp,source,log_message,target_label
0,2025-11-09 22:47:00,ModernCRM,Unauthorized login attempt on protected accoun...,Security Alert
1,2025-06-26 02:46:29,AnalyticsEngine,nova.compute.claims [req-2d658d2c-7eff-414e-a6...,Resource Usage
2,2025-02-28 23:13:12,ModernCRM,nova.osapi_compute.wsgi.server [req-7b894c1d-b...,HTTP Status
3,2025-03-28 21:45:49,ModernCRM,nova.osapi_compute.wsgi.server [req-e4a5ec58-0...,HTTP Status
4,2025-01-27 06:37:03,ModernCRM,nova.osapi_compute.wsgi.server [req-085df2be-e...,HTTP Status
...,...,...,...,...
1495,2025-06-06 15:29:00,BillingSystem,System reboot initiated by user User938.,System Notification
1496,2025-02-12 02:29:00,BillingSystem,Disk cleanup completed successfully.,System Notification
1497,5/13/2025 5:04,ThirdPartyAPI,File data_2777.csv uploaded successfully by us...,System Notification
1498,7/26/2025 2:50,ThirdPartyAPI,System updated to version 4.6.3.,System Notification


In [249]:
df.head()

,timestamp,source,log_message,target_label
0,2025-11-09 22:47:00,ModernCRM,Unauthorized login attempt on protected accoun...,Security Alert
1,2025-06-26 02:46:29,AnalyticsEngine,nova.compute.claims [req-2d658d2c-7eff-414e-a6...,Resource Usage
2,2025-02-28 23:13:12,ModernCRM,nova.osapi_compute.wsgi.server [req-7b894c1d-b...,HTTP Status
3,2025-03-28 21:45:49,ModernCRM,nova.osapi_compute.wsgi.server [req-e4a5ec58-0...,HTTP Status
4,2025-01-27 06:37:03,ModernCRM,nova.osapi_compute.wsgi.server [req-085df2be-e...,HTTP Status


In [250]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ThirdPartyAPI', 'ModernHR',
       'BillingSystem'], dtype=object)

In [251]:
df.target_label.unique()
# import sys
# print(sys.executable)

array(['Security Alert', 'Resource Usage', 'HTTP Status',
       'Critical Error', 'Error', 'System Notification', 'User Action'],
      dtype=object)

In [252]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np

# Load pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for log messages
embeddings = model.encode(df['log_message'].tolist())
embeddings[:2]

array([[-1.45718353e-02,  2.28583105e-02, -5.14790788e-02,
        -8.78430009e-02,  1.63186528e-02,  2.82762814e-02,
         8.19532871e-02, -5.49000390e-02,  8.39995034e-03,
        -2.90819574e-02,  3.19006108e-02,  2.61128936e-02,
         2.91709080e-02,  1.63090341e-02,  1.15332268e-02,
        -1.08493175e-02, -7.17150373e-03,  2.18599942e-02,
        -2.29689498e-02, -1.38238026e-03, -1.22735649e-03,
        -3.57407220e-02, -6.46761805e-02, -7.15657044e-03,
         3.22512463e-02, -2.23161560e-02,  6.28208444e-02,
         5.23375235e-02, -1.09460494e-02,  3.22110318e-02,
        -2.56337728e-02,  3.61070572e-03, -4.25966969e-03,
         3.60999280e-03,  4.81412150e-02, -4.08887081e-02,
        -1.09138750e-02,  3.50124985e-02,  3.20131741e-02,
        -9.88966376e-02, -5.95049039e-02, -2.67185997e-02,
        -4.54579946e-03,  4.62164022e-02,  1.61843933e-02,
         6.10174909e-02, -9.44626052e-03,  4.59140912e-02,
        -1.39593631e-02,  1.00315139e-02,  4.92332801e-0

In [253]:
#Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

# Add cluster labels to the DataFrame
df['cluster'] = clusters
df.head()

,timestamp,source,log_message,target_label,cluster
0,2025-11-09 22:47:00,ModernCRM,Unauthorized login attempt on protected accoun...,Security Alert,0
1,2025-06-26 02:46:29,AnalyticsEngine,nova.compute.claims [req-2d658d2c-7eff-414e-a6...,Resource Usage,1
2,2025-02-28 23:13:12,ModernCRM,nova.osapi_compute.wsgi.server [req-7b894c1d-b...,HTTP Status,2
3,2025-03-28 21:45:49,ModernCRM,nova.osapi_compute.wsgi.server [req-e4a5ec58-0...,HTTP Status,2
4,2025-01-27 06:37:03,ModernCRM,nova.osapi_compute.wsgi.server [req-085df2be-e...,HTTP Status,2


In [254]:
df[df.cluster==1]

,timestamp,source,log_message,target_label,cluster
1,2025-06-26 02:46:29,AnalyticsEngine,nova.compute.claims [req-2d658d2c-7eff-414e-a6...,Resource Usage,1
9,2025-06-27 12:54:39,BillingSystem,nova.compute.claims [req-1162e278-3bf2-4b32-93...,Resource Usage,1
29,2025-01-17 04:52:25,ThirdPartyAPI,nova.compute.claims [req-d6986b54-3735-4a42-90...,Resource Usage,1
56,2025-07-06 03:15:38,ModernCRM,nova.compute.claims [req-d82fab16-60f8-4c9f-bd...,Resource Usage,1
67,2025-08-12 19:57:05,AnalyticsEngine,nova.compute.claims [req-1162e278-3bf2-4b32-93...,Resource Usage,1
75,2025-05-25 12:11:09,ThirdPartyAPI,nova.compute.claims [req-4b4dd551-26d8-48e2-bd...,Resource Usage,1
79,2025-03-15 03:03:08,ModernHR,nova.compute.claims [req-d38f479d-9bb9-4276-96...,Resource Usage,1
116,2025-01-25 04:53:17,BillingSystem,nova.compute.claims [req-afb5ee70-39d1-435e-9b...,Resource Usage,1
167,2025-12-10 11:53:33,AnalyticsEngine,nova.compute.claims [req-97fcea79-42f7-4241-9b...,Resource Usage,1
168,2025-08-17 19:50:06,BillingSystem,nova.compute.claims [req-3d26bc3d-373c-4269-b0...,Resource Usage,1


In [255]:
#sort clusters by number of records in it. Then print 5 log messages from
# those clusters that has more than 10 records in it.
cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts > 10].index

for cluster in large_clusters:
    print(f"Cluster {cluster}:")
    print(df[df['cluster'] == cluster]['log_message'].head(5).to_string(index=False))
    print()


Cluster 2:
nova.osapi_compute.wsgi.server [req-7b894c1d-bf...
nova.osapi_compute.wsgi.server [req-e4a5ec58-03...
nova.osapi_compute.wsgi.server [req-085df2be-ea...
nova.osapi_compute.wsgi.server [req-aa55b411-2b...
nova.osapi_compute.wsgi.server [req-be61f06f-fc...

Cluster 121:
 User User827 logged in.
User User777 logged out.
User User948 logged out.
User User402 logged out.
User User459 logged out.

Cluster 122:
Backup started at 2025-03-21 00:20:21.
  Backup ended at 2025-08-18 14:29:26.
  Backup ended at 2025-08-08 13:06:23.
  Backup ended at 2025-07-31 09:29:01.
  Backup ended at 2025-03-04 01:07:19.

Cluster 123:
Backup completed successfully.
Backup completed successfully.
Backup completed successfully.
Backup completed successfully.
Backup completed successfully.

Cluster 120:
Disk cleanup completed successfully.
Disk cleanup completed successfully.
Disk cleanup completed successfully.
Disk cleanup completed successfully.
Disk cleanup completed successfully.

Cluster 126:
Syst

In [256]:
import re

def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }

    for pattern, label in regex_patterns.items():
        # Check if pattern matches log message (case-insensitive)
        if re.search(pattern, log_message, re.IGNORECASE):
            return label

    return None



In [257]:
classify_with_regex("Account with ID A0898234 created by Dhaval")

'User Action'

In [258]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df[df.regex_label.notnull()]

,timestamp,source,log_message,target_label,cluster,regex_label
800,2025-09-09 11:36:00,ModernHR,Disk cleanup completed successfully.,System Notification,120,System Notification
801,2025-09-08 01:08:00,ThirdPartyAPI,User User827 logged in.,User Action,121,User Action
802,9/14/2025 19:05,ModernCRM,User User777 logged out.,User Action,121,User Action
803,5/16/2025 5:04,AnalyticsEngine,Backup started at 2025-03-21 00:20:21.,System Notification,122,System Notification
804,3/29/2025 16:26,AnalyticsEngine,Disk cleanup completed successfully.,System Notification,120,System Notification
...,...,...,...,...,...,...
1495,2025-06-06 15:29:00,BillingSystem,System reboot initiated by user User938.,System Notification,126,System Notification
1496,2025-02-12 02:29:00,BillingSystem,Disk cleanup completed successfully.,System Notification,120,System Notification
1497,5/13/2025 5:04,ThirdPartyAPI,File data_2777.csv uploaded successfully by us...,System Notification,125,System Notification
1498,7/26/2025 2:50,ThirdPartyAPI,System updated to version 4.6.3.,System Notification,124,System Notification


In [259]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex.shape

(800, 6)

In [260]:
print(df_non_regex['target_label'].value_counts()[df_non_regex['target_label'].value_counts() <= 5].index.tolist())


[]


In [261]:
df_non_legacy = df_non_regex[df_non_regex.source != 'LegacyCRM']
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ThirdPartyAPI', 'ModernHR',
       'BillingSystem'], dtype=object)

In [ ]:
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())
filtered_embeddings[:2]

In [244]:
X = filtered_embeddings
y = df_non_legacy['target_label'].reset_index(drop=True)


In [245]:
X = np.array(X)


In [246]:
X = filtered_embeddings
y = df_non_legacy['target_label']
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)


                precision    recall  f1-score   support

Critical Error       0.94      0.94      0.94        16
         Error       0.96      0.96      0.96        28
   HTTP Status       1.00      1.00      1.00       123
Resource Usage       1.00      1.00      1.00        18
Security Alert       1.00      1.00      1.00        55

      accuracy                           0.99       240
     macro avg       0.98      0.98      0.98       240
  weighted avg       0.99      0.99      0.99       240



In [247]:
import joblib

joblib.dump(clf, '../models/log_classifier.joblib')

['../models/log_classifier.joblib']